<a href="https://colab.research.google.com/github/cleysonl/Tensorflow2.0/blob/master/Tensorflow_2_0_CC_CLL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow-gpu==2.0.0

In [3]:
import tensorflow as tf
print(tf.__version__)

2.0.0


## **1. Basics**

### **Tensors**

In [4]:
#constant tensor
x = tf.constant([[3,4],[2,1]])
print(x)

tf.Tensor(
[[3 4]
 [2 1]], shape=(2, 2), dtype=int32)


In [5]:
# Convert the value to numpy array
x.numpy()

array([[3, 4],
       [2, 1]], dtype=int32)

In [7]:
# As numpy array, it has the attribute dtype and shape:
print('dtype:', x.dtype)
print('shape:', x.shape)


dtype: <dtype: 'int32'>
shape: (2, 2)


In [8]:
# Arrays of ones an zeros
print(tf.ones(shape=(2,1)))
print(tf.zeros(shape=(2,1)))

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


### **Random constant tensors**

In [9]:
tf.random.normal(shape=(2,2),mean =0., stddev = 1.)

<tf.Tensor: id=12, shape=(2, 2), dtype=float32, numpy=
array([[ 0.24939412,  1.2122822 ],
       [ 0.22691134, -1.2168541 ]], dtype=float32)>

In [11]:
tf.random.uniform(shape=(2,2), minval=0, maxval=10, dtype='int32')

<tf.Tensor: id=19, shape=(2, 2), dtype=int32, numpy=
array([[7, 6],
       [9, 6]], dtype=int32)>

### **Variables**

In [12]:
initial_value = tf.random.normal(shape=(2,2))
a = tf.Variable(initial_value)
print(a)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[ 0.41124508,  0.11597262],
       [-1.5880194 , -1.9874033 ]], dtype=float32)>


In [18]:
new_value = tf.random.normal(shape=(2,2))
a.assign(new_value)
print(a)
for i in range(2):
  for j in range(2):
    assert a[i,j]==new_value[i,j]
print(a)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[-1.2197094 , -0.5769946 ],
       [ 0.02203372, -0.45357445]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[-1.2197094 , -0.5769946 ],
       [ 0.02203372, -0.45357445]], dtype=float32)>


In [19]:
added_value = tf.random.normal(shape=(2,2))
a.assign_add(added_value)
print(a)
for i in range(2):
  for j in range(2):
    assert a[i,j]== new_value[i,j] + added_value[i,j]
print(a)    

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[-3.984428  ,  0.4588402 ],
       [ 0.23103239,  0.65595436]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[-3.984428  ,  0.4588402 ],
       [ 0.23103239,  0.65595436]], dtype=float32)>


### **Doing math in TF**

In [0]:
a = tf.random.normal(shape=(2,2))
b = tf.random.normal(shape=(2,2))

c = a + b
d = tf.square(c)
e = tf.exp(d)

### **Computing gradient with** 'GradientTape'

In [26]:
a = tf.random.uniform(shape=(2,2))
b = tf. random.uniform(shape=(2,2))

a= tf.Variable(a)

with tf.GradientTape() as tape:
  c = tf.sqrt(tf.square(a) + tf.sqrt(b))
  dc_da = tape.gradient(c,a)
  print(dc_da)

tf.Tensor(
[[0.79893404 0.16274728]
 [0.747564   0.45697558]], shape=(2, 2), dtype=float32)


In [27]:
# High-order derivatives with nesting tapes
with tf.GradientTape() as outer_tape:
  with tf. GradientTape() as tape:
    c = tf.sqrt(tf.square(a) + tf.square(b))
    dc_da = tape.gradient(c,a)
  dc2_da2= outer_tape.gradient(dc_da,a)
  print(dc2_da2)

tf.Tensor(
[[0.00957775 1.0430269 ]
 [0.19639552 0.69217837]], shape=(2, 2), dtype=float32)
